In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import time

In [21]:
data_path='dataset/tensorized_data/'

train_data_bw=torch.load(data_path+'training_data_bw.pt').squeeze()
train_label_bw=torch.load(data_path+'training_labels_bw.pt').squeeze()
test_data_bw=torch.load(data_path+'testing_data_bw.pt').squeeze()
test_label_bw=torch.load(data_path+'testing_labels_bw.pt').squeeze()

print(train_data_bw.size(), train_label_bw.size())
print(test_data_bw.size(), test_label_bw.size())

torch.Size([3861, 28, 28]) torch.Size([3861])
torch.Size([1644, 28, 28]) torch.Size([1644])


In [22]:
mean= train_data_bw.mean()
print(mean)
std= train_data_bw.std()
print(std)

tensor(0.4084)
tensor(0.2522)


In [23]:
class convnet_bw(nn.Module):

    def __init__(self):

        super(convnet_bw, self).__init__()

        self.conv1 = nn.Conv2d(1,   50,  kernel_size=3,  padding=1 )
        
        self.pool1  = nn.MaxPool2d(2,2)
        
        self.conv2 = nn.Conv2d(50,  100,  kernel_size=3,  padding=1 )
        
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.linear1 = nn.Linear(4900, 100)
        
        self.linear2 = nn.Linear(100,2)


    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 4900)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
    
        return x

In [24]:
net=convnet_bw()
print(net)
criterion = nn.CrossEntropyLoss()
my_lr=0.25 
bs= 20

convnet_bw(
  (conv1): Conv2d(1, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(50, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=4900, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=2, bias=True)
)


In [25]:
def get_error( scores , labels ):

    bs=scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels == labels)
    num_matches=indicator.sum()
    
    return 1-num_matches.float()/bs  

In [26]:
start=time.time()

for epoch in range(30):
        
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(3860)
 
    for count in range(0,3860,bs):
        
        # FORWARD AND BACKWARD PASS
    
        optimizer.zero_grad()
             
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data_bw[indices].unsqueeze(dim=1)
        minibatch_label=  train_label_bw[indices]
    
        inputs = (minibatch_data - mean)/std      # ONLY CHANGE IS HERE!
        
        inputs.requires_grad_()

        scores=net( inputs ) 

        loss =  criterion( scores , minibatch_label) 
          
        loss.backward()
        
        optimizer.step()      
        
        running_loss += loss.detach().item()
        
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1        
    
    
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60
    
    print('epoch=',epoch, '\t time=', elapsed,'min', '\t lr=', my_lr  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    print(' ')
    

epoch= 0 	 time= 0.09891424179077149 min 	 lr= 0.25 	 loss= 0.6562244282176457 	 error= 39.378238280202446 percent
 
epoch= 1 	 time= 0.19587635199228923 min 	 lr= 0.25 	 loss= 0.6346996926890753 	 error= 36.86528486909027 percent
 
epoch= 2 	 time= 0.29737677971522014 min 	 lr= 0.25 	 loss= 0.6217288574097688 	 error= 34.48186541468368 percent
 
epoch= 3 	 time= 0.39569318294525146 min 	 lr= 0.25 	 loss= 0.6066072112229204 	 error= 33.31606233675863 percent
 
epoch= 4 	 time= 0.502450184027354 min 	 lr= 0.25 	 loss= 0.5883514840985827 	 error= 32.720207365065654 percent
 
epoch= 5 	 time= 0.5934579292933146 min 	 lr= 0.25 	 loss= 0.5738419998495072 	 error= 29.48186548880345 percent
 
epoch= 6 	 time= 0.6834574540456136 min 	 lr= 0.25 	 loss= 0.5398917236785197 	 error= 27.35751277424511 percent
 
epoch= 7 	 time= 0.771706759929657 min 	 lr= 0.25 	 loss= 0.4985937402038377 	 error= 24.896372846988815 percent
 
epoch= 8 	 time= 0.8611282070477804 min 	 lr= 0.25 	 loss= 0.43752050025321

In [31]:
net.eval()
with torch.no_grad():
    data = test_data_bw.unsqueeze(dim=1)
    labels = test_label_bw
    scores = net(data) 
    print("Test error={} percent".format(get_error(scores,labels)*100))

Test error=40.936737060546875 percent


In [32]:
train_data_color=torch.load(data_path+'training_data_color.pt').squeeze()
train_label_color=torch.load(data_path+'training_labels_color.pt').squeeze()
test_data_color=torch.load(data_path+'testing_data_color.pt').squeeze()
test_label_color=torch.load(data_path+'testing_labels_color.pt').squeeze()

print(train_data_color.size(), train_label_color.size())
print(test_data_color.size(), test_label_color.size())

torch.Size([3860, 3, 28, 28]) torch.Size([3860])
torch.Size([1645, 3, 28, 28]) torch.Size([1645])


In [33]:
mean= train_data_color.mean()
print(mean)
std= train_data_color.std()
print(std)

tensor(0.4060)
tensor(0.2611)


In [34]:
class convnet_color(nn.Module):

    def __init__(self):

        super(convnet_color, self).__init__()

        self.conv1 = nn.Conv2d(3,   50,  kernel_size=3,  padding=1 )
        
        self.pool1  = nn.MaxPool2d(2,2)
        
        self.conv2 = nn.Conv2d(50,  100,  kernel_size=3,  padding=1 )
        
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.linear1 = nn.Linear(4900, 100)
        
        self.linear2 = nn.Linear(100,2)


    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 4900)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
    
        return x

In [37]:
net_color=convnet_color()
print(net_color)
criterion = nn.CrossEntropyLoss()
my_lr=0.25 
bs= 20

convnet_color(
  (conv1): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(50, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=4900, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=2, bias=True)
)


In [39]:
start=time.time()

for epoch in range(30):
        
    optimizer=torch.optim.SGD( net_color.parameters() , lr=my_lr )
        
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(3860)
 
    for count in range(0,3860,bs):
        
        # FORWARD AND BACKWARD PASS
    
        optimizer.zero_grad()
             
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data_color[indices]
        minibatch_label=  train_label_color[indices]
    
        inputs = (minibatch_data - mean)/std    
        
        inputs.requires_grad_()

        scores=net_color( inputs ) 

        loss =  criterion( scores , minibatch_label) 
          
        loss.backward()
        
        optimizer.step()      
        
        running_loss += loss.detach().item()
        
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1        
    
    
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60
    
    print('epoch=',epoch, '\t time=', elapsed,'min', '\t lr=', my_lr  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    print(' ')
    

epoch= 0 	 time= 0.10254774490992229 min 	 lr= 0.25 	 loss= 0.6581064099783724 	 error= 40.49222818310397 percent
 
epoch= 1 	 time= 0.2047592600186666 min 	 lr= 0.25 	 loss= 0.6273813693943419 	 error= 36.606217388044364 percent
 
epoch= 2 	 time= 0.3191284974416097 min 	 lr= 0.25 	 loss= 0.596043035465201 	 error= 34.404145034483676 percent
 
epoch= 3 	 time= 0.42218162218729655 min 	 lr= 0.25 	 loss= 0.5772449942450449 	 error= 31.217616672960585 percent
 
epoch= 4 	 time= 0.5312346935272216 min 	 lr= 0.25 	 loss= 0.5575364188054682 	 error= 30.336787663593196 percent
 
epoch= 5 	 time= 0.6471272150675456 min 	 lr= 0.25 	 loss= 0.5359665918998767 	 error= 29.352331686514027 percent
 
epoch= 6 	 time= 0.7490355372428894 min 	 lr= 0.25 	 loss= 0.507871785389327 	 error= 25.336787552413547 percent
 
epoch= 7 	 time= 0.8537233710289002 min 	 lr= 0.25 	 loss= 0.46856688542069547 	 error= 23.26424832171109 percent
 
epoch= 8 	 time= 0.9613411029179891 min 	 lr= 0.25 	 loss= 0.411966690436

In [41]:
net_color.eval()
with torch.no_grad():
    data = test_data_color
    labels = test_label_color
    scores = net_color(data) 
    print("Test error={} percent".format(get_error(scores,labels)*100))

Test error=39.878421783447266 percent
